In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 10000
N_bound = 200

# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)
x_inside = traindata[:,0:1]
t_inside = traindata[:,1:2]

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
t_inside = numpy_to_tensor(t_inside, var_name="t_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

In [3]:
import scipy.io
data = scipy.io.loadmat('./burgers_shock.mat')
t_exact = data['t'].flatten()[:,None]
x_exact = data['x'].flatten()[:,None]
Exact_u = np.real(data['usol'])

In [ ]:
print('t:',t_exact.shape)
print('x:',x_exact.shape)
print('Exact_u:',Exact_u.shape)

In [ ]:
T_exact, X_exact = np.meshgrid(t_exact,x_exact)
print('T_exact:',T_exact.shape)
print('X_exact:',X_exact.shape)

In [6]:
T_exact_flatten = T_exact.flatten()[:,None]
X_exact_flatten = X_exact.flatten()[:,None]
data_star = np.hstack((X_exact_flatten,T_exact_flatten))
u_star = Exact_u.flatten()[:,None]  

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(6, 2))

cmap = plt.get_cmap('jet')  
heatmap1 = ax1.imshow(Exact_u, interpolation='nearest', cmap='rainbow',  extent=[0, 1, -1, 1], origin='lower', aspect='auto')
ax1.set_title('u', fontsize=15)
ax1.set_xlabel(r'$t$', fontsize=15)
ax1.set_ylabel(r'$x$', fontsize=15)

# Add a color bar
cbar = fig.colorbar(heatmap1, ax=ax1, fraction=0.15, pad=0.04)
cbar.set_label('Value', fontsize=12)

plt.show()

In [ ]:
random_seed = 1234
np.random.seed(random_seed)
number_observe_data = 100

index_t = (lhs(1,number_observe_data))*len(t_exact)
index_t = np.floor(index_t).reshape(number_observe_data,).astype(int)
observe_t = t_exact[index_t]

index_x = (lhs(1,number_observe_data))*len(x_exact)
index_x = np.floor(index_x).reshape(number_observe_data,).astype(int)
observe_x = x_exact[index_x]

observe_data = np.hstack((observe_x,observe_t))
observe_clear_u = Exact_u[index_x,index_t].reshape(-1,1)

############# N(0,0.1^2) #############
noise_nu = 0
noise_std = 0.1
noise_u = np.random.normal(loc=noise_nu, scale=noise_std, size=observe_clear_u.shape)
observe_u = observe_clear_u + noise_u
############# N(0,0.1^2) #############

observe_data = numpy_to_tensor(observe_data, var_name="observe_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_clear_u = numpy_to_tensor(observe_clear_u, var_name="observe_clear_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_u = numpy_to_tensor(observe_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
print('J:',len(observe_u))

observe_data_x_inside = observe_data[:,0:1]
observe_data_t_inside = observe_data[:,1:2]

In [ ]:
np.random.seed(5678)
n_test_data = 10000

index_test_data = (lhs(1,n_test_data))*len(data_star)
index_test_data = np.floor(index_test_data).reshape(n_test_data,).astype(int)

index_t = (lhs(1,n_test_data))*len(t_exact)
index_t = np.floor(index_t).reshape(n_test_data,).astype(int)
test_data_t = t_exact[index_t]

index_x = (lhs(1,n_test_data))*len(x_exact)
index_x = np.floor(index_x).reshape(n_test_data,).astype(int)
test_data_x = x_exact[index_x]

test_data = np.hstack((test_data_x,test_data_t))
test_u = Exact_u[index_x,index_t].reshape(-1,1)

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
test_u = numpy_to_tensor(test_u, var_name="test_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

test_data_x_inside = test_data[:,0:1]
test_data_t_inside = test_data[:,1:2]

In [10]:
def output_transform(data_input, u_input):
    
    x_in = data_input[:, 0:1]
    t_in = data_input[:, 1:2]

    return (1 - x_in) * (1 + x_in) * (1 - torch.exp(-t_in)) * u_input - torch.sin(np.pi * x_in)

In [11]:
C1 = torch.tensor(0.5, requires_grad=True)

In [12]:
def get_loss_f(x_grad,t_grad,PINNs,C,return_sequence='not'):
    
    ########### loss f  ###########
    E_inside = PINNs(torch.cat((x_grad,t_grad),1)) 
    E_inside = output_transform(torch.cat((x_grad,t_grad),1),E_inside) 
    
    E_x = compute_higher_order_derivatives(E_inside, [x_grad])
    E_xx = compute_higher_order_derivatives(E_x, [x_grad])
    E_t = compute_higher_order_derivatives(E_inside, [t_grad])


    if return_sequence=='yes':
        return torch.square(E_t+E_inside*E_x-C*(E_xx))
    else:
        return torch.mean(torch.square(E_t+E_inside*E_x-C*(E_xx)))

In [13]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [14]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

net_settings_for_PINNs1 = NetSetting(input_dims=2, hidden_neurons_list=[20]*8, 
                                     output_dims=1, hidden_activation='tanh', 
                                     output_activation=None, initializer_method='xavier')

PINNs1 = get_mlp_pinn(net_settings_for_PINNs1)
PINNs1.cuda()  

initialize_weights(PINNs1, net_settings_for_PINNs1.initializer_method)
   
optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)    
optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

In [15]:
x_inside_all = torch.cat((x_inside,observe_data[:,0:1]),axis=0)
t_inside_all = torch.cat((t_inside,observe_data[:,1:2]),axis=0)

In [ ]:
############## Record list ###############
loss_all_1 = []
loss_f_1 = []
loss_f_for_collocation_1 = []
loss_f_for_T_1 = []
loss_f_excapt_T_1 = []
loss_T_1 = []
loss_T_clear_1 = []
loss_T_1_test_data = []
test_loss_1 = []
C1_list = []
############## Record list ###############


# C1 = torch.tensor(0.01/np.pi, requires_grad=False)
nIter1 = 20000
it = 0

while it<nIter1:
   
    #########loss f #########
    loss_f = get_loss_f(x_inside_all,t_inside_all,PINNs1,C1, return_sequence='not')
    
    #########loss f  for collocation data#########
    loss_f_for_collocation = get_loss_f(x_inside,t_inside,PINNs1,C1,return_sequence='not')
    #########loss f  for observation data#########
    loss_f_for_T = get_loss_f(observe_data_x_inside,observe_data_t_inside,PINNs1,C1,return_sequence='not')
    #########loss f  excapt observation data#########
    loss_f_excapt_T = get_loss_f(test_data_x_inside,test_data_t_inside,PINNs1,C1, return_sequence='not')
    
    #########loss T observation#########        
    E_observation = PINNs1(observe_data) 
    E_observation = output_transform(observe_data,E_observation) 
    loss_observation = torch.mean(torch.square(E_observation-observe_u))            
    
    #########loss T noisy observation#########  
    loss_observation_noisy = torch.mean(torch.square(E_observation-observe_u)) 
    
    #########loss T clear observation#########        
    E_observation_clear = PINNs1(observe_data) 
    E_observation_clear = output_transform(observe_data,E_observation_clear) 
    loss_observation_clear = torch.mean(torch.square(E_observation_clear-observe_clear_u))        
    
    #########loss T excapt observation#########        
    E_observation_excapt = PINNs1(test_data) 
    E_observation_excapt = output_transform(test_data,E_observation_excapt) 
    loss_observation_excapt = torch.mean(torch.square(E_observation_excapt-test_u)) 
    
    #########loss PI#########
    loss = loss_f+10*loss_observation

    #########test_loss NRMSE#########
    pre_u = PINNs1(test_data)
    pre_u = output_transform(test_data,pre_u) 
    test_loss = relative_l2_torch(pre_u,test_u)
    #########test_loss NRMSE#########
    
    #########Record#########
    loss_f_1.append(loss_f.item())
    loss_f_for_collocation_1.append(loss_f_for_collocation.item())
    loss_f_for_T_1.append(loss_f_for_T.item())
    loss_f_excapt_T_1.append(loss_f_excapt_T.item())
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation_noisy.item()) 
    loss_T_clear_1.append(loss_observation_clear.item()) 
    loss_T_1_test_data.append(loss_observation_excapt.item()) 
    test_loss_1.append(test_loss)
    #########Record#########
        
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
       
    if it % 1000 == 0:

        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss)
        print('C1',C1)   
        
        
    it = it + 1        

print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)